<img width="800px" src="../fidle/img/00-Fidle-header-01.svg"></img>

# <!-- TITLE --> [AE3] - Playing with our denoiser model
<!-- DESC --> Episode 2 : Using the previously trained autoencoder to denoise data
<!-- AUTHOR : Jean-Luc Parouty (CNRS/SIMaP) -->

## Objectives :
 - Retrieve and use our denoiser model


## What we're going to do :

 - Reload our dataset and saved best model
 - Encode/decode some test images (neved used, never seen by the model)
 
## Data Terminology :
- `clean_train`, `clean_test` for noiseless images 
- `noisy_train`, `noisy_test` for noisy images
- `denoised_test` for denoised images at the output of the model


## Step 1 - Init python stuff
### 1.1 - Init

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import h5py
import random

import tensorflow as tf
from tensorflow import keras

from modules.MNIST import MNIST

import fidle

# Init Fidle environment
run_id, run_dir, datasets_dir = fidle.init('AE3')

### 1.2 - Parameters
These **parameters must be identical** to those used during the training in order to have the **same dataset**.\
`prepared_dataset` : Filename of the prepared dataset (Need 400 Mo, but can be in ./data)  
`dataset_seed` : Random seed for shuffling dataset  
`scale` : % of the dataset to use (1. for 100%)  
`train_prop` : Percentage for train (the rest being for the test)

In [ ]:
prepared_dataset = './data/mnist-noisy.h5'
saved_models     = './run/AE2/models'
dataset_seed     = 123
scale            = 1
train_prop       = .8

Override parameters (batch mode) - Just forget this cell

In [ ]:
fidle.override('prepared_dataset', 'dataset_seed', 'scale', 'train_prop')

## Step 2 - Retrieve dataset
With our MNIST class, in one call, we can reload, rescale, shuffle and split our previously saved dataset :-)  
**Important :** Make sure that the **digest is identical** to the one used during the training !\
See : [AE2 / Step 2 - Retrieve dataset](./02-AE-with-MNIST.ipynb#Step-2---Retrieve-dataset)

In [ ]:
clean_train,clean_test, noisy_train,noisy_test, _,_ = MNIST.reload_prepared_dataset(scale      = scale, 
                                                                                    train_prop = train_prop,
                                                                                    seed       = dataset_seed,
                                                                                    shuffle    = True,
                                                                                    filename=prepared_dataset )

## Step 3 - Evaluation
**Note :** We will use the following data:\
`clean_train`, `clean_test` for noiseless images \
`noisy_train`, `noisy_test` for noisy images\
`denoised_test` for denoised images at the output of the model
 
### 3.1 - Reload our best model

In [ ]:
model = keras.models.load_model(f'{saved_models}/best_model.h5')

### 3.2 - Let's make a prediction

In [ ]:
from tabnanny import verbose


denoised_test = model.predict(noisy_test,verbose=0)

print('Denoised images   (denoised_test) shape : ',denoised_test.shape)

### 3.3 - Denoised images 

In [ ]:
i=random.randint(0,len(denoised_test)-8)
j=i+8

fidle.utils.subtitle('Noisy test images (input):')
fidle.scrawler.images(noisy_test[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='05-test-noisy')

fidle.utils.subtitle('Denoised images (output):')
fidle.scrawler.images(denoised_test[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='06-test-predict')

fidle.utils.subtitle('Real test images :')
fidle.scrawler.images(clean_test[i:j], None, indices='all', columns=8, x_size=2,y_size=2, interpolation=None, save_as='07-test-real')

## Step 4 - Looking at the latent space
### 4.1 - Getting clean data and class

In [ ]:
clean_data,_, _,_, class_data,_ = MNIST.reload_prepared_dataset(scale      = 1, 
                                                                train_prop = 1,
                                                                seed       = dataset_seed,
                                                                shuffle    = False,
                                                                filename   = prepared_dataset )

### 4.2 - Retrieve encoder

In [ ]:
encoder=model.get_layer('encoder')

### 4.3 Showing latent space
Here is the digit distribution in the latent space

In [ ]:
n_show = 20000

# ---- Select images

x_show, y_show = fidle.utils.pick_dataset(clean_data, class_data, n=n_show)

# ---- Get latent points

z = encoder.predict(x_show)

# ---- Show them

fig = plt.figure(figsize=(14, 10))
plt.scatter(z[:, 0] , z[:, 1], c=y_show, cmap= 'tab10', alpha=0.5, s=30)
plt.colorbar()
fidle.scrawler.save_fig('08-Latent-space')
plt.show()

In [ ]:
fidle.end()

---
<img width="80px" src="../fidle/img/00-Fidle-logo-01.svg"></img>